In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pycaret
from pycaret.classification import *
from pycaret.datasets import get_data

In [59]:
data = pd.read_csv('train.csv')
new_cols = ['Date','Time','PM25','PM10','O3','CO','NO2','SO2','WS','TEMP','RH','WD']
data = data[new_cols]
data

,Date,Time,PM25,PM10,O3,CO,NO2,SO2,WS,TEMP,RH,WD
0,2024-01-01,00:00:00,11.5,20.0,23.00,0.20,1.00,0.00,2,28.0,86,15
1,2024-01-01,01:00:00,11.2,18.0,22.00,0.27,2.00,0.00,2,27.0,88,352
2,2024-01-01,02:00:00,9.8,17.0,19.00,0.26,3.00,0.00,2,27.0,90,348
3,2024-01-01,03:00:00,9.7,17.0,17.00,0.26,2.00,0.00,2,26.0,91,1
4,2024-01-01,04:00:00,8.7,16.0,17.00,0.26,2.00,0.00,2,27.0,90,3
...,...,...,...,...,...,...,...,...,...,...,...,...
1138,2024-02-18,06:00:00,15.6,32.0,14.00,0.05,3.00,0.00,1,25.0,85,30
1139,2024-02-18,07:00:00,17.7,37.0,7.00,0.07,2.17,0.00,0,25.0,84,274
1140,2024-02-18,08:00:00,21.4,48.0,18.00,0.20,2.00,0.15,2,29.0,64,56
1141,2024-02-18,09:00:00,16.3,41.0,23.52,0.16,1.00,0.00,4,31.0,57,60


In [97]:
pm25_for_predict = (data.drop(columns=['WS', 'TEMP', 'RH', 'WD']))
pm25_for_predict

,Date,Time,PM25,PM10,O3,CO,NO2,SO2
0,2024-01-01,00:00:00,11.5,20.0,23.00,0.20,1.00,0.00
1,2024-01-01,01:00:00,11.2,18.0,22.00,0.27,2.00,0.00
2,2024-01-01,02:00:00,9.8,17.0,19.00,0.26,3.00,0.00
3,2024-01-01,03:00:00,9.7,17.0,17.00,0.26,2.00,0.00
4,2024-01-01,04:00:00,8.7,16.0,17.00,0.26,2.00,0.00
...,...,...,...,...,...,...,...,...
1138,2024-02-18,06:00:00,15.6,32.0,14.00,0.05,3.00,0.00
1139,2024-02-18,07:00:00,17.7,37.0,7.00,0.07,2.17,0.00
1140,2024-02-18,08:00:00,21.4,48.0,18.00,0.20,2.00,0.15
1141,2024-02-18,09:00:00,16.3,41.0,23.52,0.16,1.00,0.00


In [3]:
from pycaret.regression import *
clf1 = setup(data, target='PM25', session_id = 123)

,Description,Value
0,Session id,123
1,Target,PM25
2,Target type,Regression
3,Original data shape,"(1143, 10)"
4,Transformed data shape,"(1143, 10)"
5,Transformed train set shape,"(800, 10)"
6,Transformed test set shape,"(343, 10)"
7,Numeric features,9
8,Preprocess,True
9,Imputation type,simple


In [4]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,2.0834,10.0002,3.1394,0.8043,0.1680,0.1289,0.0530
gbr,Gradient Boosting Regressor,2.1099,10.0654,3.1533,0.8036,0.1690,0.1307,0.0150
et,Extra Trees Regressor,2.0846,10.2516,3.1828,0.7992,0.1691,0.1301,0.0280
rf,Random Forest Regressor,2.1549,10.7180,3.2543,0.7893,0.1709,0.1335,0.0340
lar,Least Angle Regression,2.6008,14.3823,3.7617,0.7117,0.2037,0.1664,0.0040
lr,Linear Regression,2.6037,14.3918,3.7629,0.7116,0.2038,0.1666,0.2630
ridge,Ridge Regression,2.5979,14.4021,3.7647,0.7113,0.2036,0.1663,0.0040
br,Bayesian Ridge,2.5966,14.5646,3.7880,0.7080,0.2047,0.1662,0.0040
en,Elastic Net,2.7171,15.3545,3.8933,0.6932,0.2081,0.1733,0.0050
ada,AdaBoost Regressor,3.0335,15.9382,3.9740,0.6877,0.2207,0.2007,0.0120


In [5]:
tuned_et = tune_model(best)
bagged = ensemble_model(tuned_et , n_estimators=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2504,9.2242,3.0371,0.8246,0.1667,0.1400
1,2.2237,9.0184,3.0031,0.7821,0.1669,0.1402
2,2.3649,11.0983,3.3314,0.7706,0.1784,0.1404
3,2.5446,11.8201,3.4380,0.7250,0.1960,0.1708
4,2.2906,11.7238,3.4240,0.7646,0.1860,0.1495
5,2.7993,17.5512,4.1894,0.7085,0.1955,0.1690
6,2.5368,11.5751,3.4022,0.7792,0.1823,0.1552
7,2.3630,11.6215,3.4090,0.7751,0.1989,0.1537
8,2.7719,15.4847,3.9351,0.7509,0.2043,0.1557


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.0012,8.0380,2.8351,0.8471,0.1593,0.1249
1,2.1060,9.5269,3.0866,0.7698,0.1630,0.1314
2,2.1477,9.6971,3.1140,0.7996,0.1674,0.1285
3,2.0000,9.8703,3.1417,0.7704,0.1630,0.1290
4,1.9672,9.4324,3.0712,0.8106,0.1644,0.1242
5,2.5040,15.1249,3.8891,0.7488,0.1827,0.1512
6,2.0056,8.7360,2.9557,0.8333,0.1511,0.1174
7,1.7933,8.6448,2.9402,0.8327,0.1675,0.1141
8,2.1668,13.0078,3.6066,0.7907,0.1882,0.1282


In [6]:
test = pd.read_csv('test.csv')
predict_model(bagged,data=test)

,PM10,O3,CO,NO2,SO2,WS,TEMP,RH,WD,PM25,prediction_label
0,32.0,24.00,0.13,1.0,0.00,5,33.0,53,48,15.9,15.933494
1,33.0,27.00,0.14,1.0,0.00,5,33.0,51,49,12.6,14.060483
2,39.0,27.00,0.13,1.0,0.00,4,34.0,50,49,10.1,15.668284
3,34.0,28.00,0.13,1.0,0.00,5,33.0,51,52,8.7,14.565188
4,32.0,23.00,0.14,1.0,0.00,6,33.0,55,56,12.1,14.352111
...,...,...,...,...,...,...,...,...,...,...,...
296,22.0,16.00,0.20,2.0,0.15,4,29.0,67,56,6.0,10.168363
297,21.0,23.52,0.11,2.0,0.00,3,28.0,73,46,8.4,10.291777
298,23.0,14.00,0.10,1.0,0.00,3,27.0,75,54,11.7,12.885342
299,29.0,14.00,0.10,1.0,0.00,3,27.0,77,43,9.2,15.827372


In [7]:
lightgbm = create_model('lightgbm')
save_model(lightgbm, 'lightgbm_pipeline')
load_model('lightgbm_pipeline')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.0516,8.1166,2.8490,0.8456,0.1625,0.1284
1,2.0267,8.8298,2.9715,0.7867,0.1590,0.1259
2,2.2241,9.6220,3.1019,0.8011,0.1692,0.1319
3,2.2038,10.9462,3.3085,0.7453,0.1709,0.1379
4,1.9411,10.4053,3.2257,0.7910,0.1685,0.1211
5,2.4096,14.4047,3.7954,0.7607,0.1829,0.1467
6,2.1572,9.0751,3.0125,0.8269,0.1584,0.1280
7,1.9023,9.3634,3.0600,0.8188,0.1721,0.1198
8,2.2581,13.5103,3.6756,0.7826,0.1936,0.1360


Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=C:\Users\student\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['PM10', 'O3', 'CO', 'NO2', 'SO2',
                                             'WS', 'TEMP', 'RH', 'WD'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('trained_model', LGBMRegressor(n_jobs=-1, random_state=123))])

In [8]:
pre1 = predict_model(bagged,data=data)
pre1

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,1.3786,5.0138,2.2391,0.8998,0.1220,0.0866


,PM10,O3,CO,NO2,SO2,WS,TEMP,RH,WD,PM25,prediction_label
0,20.0,23.00,0.20,1.00,0.00,2,28.0,86,15,11.500000,10.687647
1,18.0,22.00,0.27,2.00,0.00,2,27.0,88,352,11.200000,10.529532
2,17.0,19.00,0.26,3.00,0.00,2,27.0,90,348,9.800000,9.902279
3,17.0,17.00,0.26,2.00,0.00,2,26.0,91,1,9.700000,9.579016
4,16.0,17.00,0.26,2.00,0.00,2,27.0,90,3,8.700000,8.990121
...,...,...,...,...,...,...,...,...,...,...,...
1138,32.0,14.00,0.05,3.00,0.00,1,25.0,85,30,15.600000,16.217356
1139,37.0,7.00,0.07,2.17,0.00,0,25.0,84,274,17.700001,18.146615
1140,48.0,18.00,0.20,2.00,0.15,2,29.0,64,56,21.400000,22.711431
1141,41.0,23.52,0.16,1.00,0.00,4,31.0,57,60,16.299999,18.088380


In [9]:
pre2 = predict_model(bagged,data=test)
pre2

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,3.5597,18.4021,4.2898,0.1937,0.3524,0.3968


,PM10,O3,CO,NO2,SO2,WS,TEMP,RH,WD,PM25,prediction_label
0,32.0,24.00,0.13,1.0,0.00,5,33.0,53,48,15.9,15.933494
1,33.0,27.00,0.14,1.0,0.00,5,33.0,51,49,12.6,14.060483
2,39.0,27.00,0.13,1.0,0.00,4,34.0,50,49,10.1,15.668284
3,34.0,28.00,0.13,1.0,0.00,5,33.0,51,52,8.7,14.565188
4,32.0,23.00,0.14,1.0,0.00,6,33.0,55,56,12.1,14.352111
...,...,...,...,...,...,...,...,...,...,...,...
296,22.0,16.00,0.20,2.0,0.15,4,29.0,67,56,6.0,10.168363
297,21.0,23.52,0.11,2.0,0.00,3,28.0,73,46,8.4,10.291777
298,23.0,14.00,0.10,1.0,0.00,3,27.0,75,54,11.7,12.885342
299,29.0,14.00,0.10,1.0,0.00,3,27.0,77,43,9.2,15.827372
